In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import time
from sklearn.metrics import accuracy_score

In [2]:
label_df = pd.read_csv(r'label.csv', header = None)
data_df = pd.read_csv(r'data.csv', header = None)

In [3]:
label_df.head()

,0
0,7
1,2
2,1
3,0
4,4


In [4]:
data_df.head()

,0,1,2,3,4,5,6,7,8,9,...,774,775,776,777,778,779,780,781,782,783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Q1

Q1: Run K-means clustering with Euclidean, Cosine and Jarcard similarity. Specify K= the
number of categorical values of y (the number of classifications). Compare the SSEs of
Euclidean-K-means, Cosine-K-means, Jarcard-K-means. Which method is better?

In [5]:
X = data_df.values.astype(float)
y = label_df.values.squeeze()
K = len(np.unique(y))

In [6]:
print(K)

10


In [7]:
def compute_distances(X, centroids, metric):
    n, d = X.shape
    k = centroids.shape[0]
    dists = np.empty((n, k))
    if metric == 'euclidean':
        for i in range(k):
            diff = X - centroids[i]
            dists[:, i] = np.sum(diff ** 2, axis=1) ** 0.5
    elif metric == 'cosine':
        X_norm = np.linalg.norm(X, axis=1)
        C_norm = np.linalg.norm(centroids, axis=1)
        sim = X @ centroids.T / (X_norm[:, None] * C_norm[None, :])
        dists = 1.0 - sim
    elif metric == 'jaccard':
        for i in range(k):
            mn = np.minimum(X, centroids[i])
            mx = np.maximum(X, centroids[i])
            sim = mn.sum(axis=1) / (mx.sum(axis=1))
            dists[:, i] = 1.0 - sim
    return dists

def kmeans(X, K, metric='euclidean', max_iter=500, random_state=42):
    rng = np.random.RandomState(random_state)
    idx = rng.choice(X.shape[0], K, replace=False)
    centroids = X[idx].copy()
    for it in range(max_iter):
        dists = compute_distances(X, centroids, metric)
        labels = np.argmin(dists, axis=1)
        new_centroids = []
        for k in range(K):
            cluster_points = X[labels == k]
            if len(cluster_points) == 0:
                new_centroids.append(centroids[k])
            else:
                new_centroids.append(cluster_points.mean(axis=0))
        new_centroids = np.vstack(new_centroids)
        if np.allclose(centroids, new_centroids):
            break
        centroids = new_centroids
    dists = compute_distances(X, centroids, metric)
    chosen = dists[np.arange(X.shape[0]), labels]
    sse = np.sum(chosen ** 2)
    return centroids, labels, sse

In [8]:
centroids_eu, labels_eu, sse_eu = kmeans(X, K, metric='euclidean')
centroids_co, labels_co, sse_co = kmeans(X, K, metric='cosine')
centroids_ja, labels_ja, sse_ja = kmeans(X, K, metric='jaccard')

In [9]:
print('Euclidean K-means SSE:', sse_eu)
print('Cosine K-means SSE:', sse_co)
print('Jaccard K-means SSE:', sse_ja)

Euclidean K-means SSE: 25414767689.961178
Cosine K-means SSE: 686.4355725684936
Jaccard K-means SSE: 3660.3894937165674


### Q2

Q2: Compare the accuracies of Euclidean-K-means Cosine-K-means, Jarcard-K-means. First,
label each cluster using the majority vote label of the data points in that cluster. Later, compute
the predictive accuracy of Euclidean-K-means, Cosine-K-means, Jarcard-K-means. Which metric
is better?

In [10]:
def majority_vote(labels_pred, y_true, K):
    mapping = {}
    for k in range(K):
        cluster_indices = np.where(labels_pred == k)[0]
        if len(cluster_indices) == 0:
            mapping[k] = -1
        else:
            true_labels = y_true[cluster_indices]
            majority_label = Counter(true_labels).most_common(1)[0][0]
            mapping[k] = majority_label
    return mapping

In [11]:
def map_cluster_to_label(labels_pred, mapping):
    return np.array([mapping[c] for c in labels_pred])

In [12]:
def compute_accuracy(labels_pred, y_true, K):
    mapping = majority_vote(labels_pred, y_true, K)
    mapped_labels = map_cluster_to_label(labels_pred, mapping)
    acc = accuracy_score(y_true, mapped_labels)
    return acc

In [13]:
acc_eu = compute_accuracy(labels_eu, y, K)
acc_co = compute_accuracy(labels_co, y, K)
acc_ja = compute_accuracy(labels_ja, y, K)

In [14]:
print("Euclidean K-means Accuracy:", acc_eu)
print("Cosine K-means Accuracy:", acc_co)
print("Jaccard K-means Accuracy:", acc_ja)

Euclidean K-means Accuracy: 0.5851
Cosine K-means Accuracy: 0.6309
Jaccard K-means Accuracy: 0.6021


### Q3

Q3: Set up the same stop criteria: “when there is no change in centroid position OR when the
SSE value increases in the next iteration OR when the maximum preset value (e.g., 500, you
can set the preset value by yourself) of iteration is complete”, for Euclidean-K-means, Cosine-Kmeans,
Jarcard-K-means. Which method requires more iterations and times to converge?

In [15]:
def kmeans_q3(X, K, metric='euclidean', max_iter=500, random_state=42):
    start = time.time()

    rng = np.random.RandomState(random_state)
    idx = rng.choice(X.shape[0], K, replace=False)
    centroids = X[idx].copy()

    prev_sse = None

    for it in range(max_iter):
        dists = compute_distances(X, centroids, metric)
        labels = np.argmin(dists, axis=1)

        chosen = dists[np.arange(X.shape[0]), labels]
        sse = np.sum(chosen ** 2)

        if prev_sse is not None and sse > prev_sse:
            break
        prev_sse = sse

        new_centroids = []
        for k in range(K):
            cluster_points = X[labels == k]
            if len(cluster_points) == 0:
                new_centroids.append(centroids[k])
            else:
                new_centroids.append(cluster_points.mean(axis=0))
        new_centroids = np.vstack(new_centroids)

        if np.allclose(centroids, new_centroids):
            centroids = new_centroids
            break

        centroids = new_centroids

    end = time.time()
    elapsed = end - start

    return centroids, labels, sse, it+1, elapsed

In [16]:
ce, le, se, ite, timee = kmeans_q3(X, K, metric='euclidean')
cc, lc, sc, itc, timec = kmeans_q3(X, K, metric='cosine')
cj, lj, sj, itj, timej = kmeans_q3(X, K, metric='jaccard')

print("Euclidean: iterations =", ite, ", time =", timee)
print("Cosine: iterations =", itc, ", time =", timec)
print("Jaccard: iterations =", itj, ", time =", timej)

Euclidean: iterations = 33 , time = 13.966703653335571
Cosine: iterations = 29 , time = 2.5885190963745117
Jaccard: iterations = 2 , time = 0.8552865982055664


### Q4

Q4: Compare the SSEs of Euclidean-K-means Cosine-K-means, Jarcard-K-means with respect to
the following three terminating conditions: (10 points)
• when there is no change in centroid position
• when the SSE value increases in the next iteration
• when the maximum preset value (e.g., 100) of iteration is complete

In [17]:
def kmeans_q4(X, K, metric='euclidean', max_iter=100, mode='centroid', random_state=42):
    rng = np.random.RandomState(random_state)
    idx = rng.choice(X.shape[0], K, replace=False)
    centroids = X[idx].copy()

    prev_sse = None

    for it in range(max_iter):
        dists = compute_distances(X, centroids, metric)
        labels = np.argmin(dists, axis=1)

        chosen = dists[np.arange(X.shape[0]), labels]
        sse = np.sum(chosen ** 2)

        if mode == 'sse_increase':
            if prev_sse is not None and sse > prev_sse:
                break
            prev_sse = sse

        new_centroids = []
        for k in range(K):
            cluster_points = X[labels == k]
            if len(cluster_points) == 0:
                new_centroids.append(centroids[k])
            else:
                new_centroids.append(cluster_points.mean(axis=0))
        new_centroids = np.vstack(new_centroids)

        if mode == 'centroid':
            if np.allclose(centroids, new_centroids):
                centroids = new_centroids
                break

        centroids = new_centroids

        if mode == 'max_iter':
            pass

    dists = compute_distances(X, centroids, metric)
    labels = np.argmin(dists, axis=1)
    chosen = dists[np.arange(X.shape[0]), labels]
    final_sse = np.sum(chosen ** 2)

    return centroids, labels, final_sse, it + 1

In [18]:
metrics = ['euclidean', 'cosine', 'jaccard']
modes = ['centroid', 'sse_increase', 'max_iter']

results = {}

for mode in modes:
    print(f'\n=== Mode: {mode} ===')
    for metric in metrics:
        cent, lab, sse, iters = kmeans_q4(X, K, metric=metric, max_iter=100, mode=mode, random_state=42)
        results[(mode, metric)] = (sse, iters)
        print(f'{metric:10s} -> SSE = {sse:.4f}, iters = {iters}')


=== Mode: centroid ===
euclidean  -> SSE = 25414767689.9612, iters = 33
cosine     -> SSE = 686.4356, iters = 48
jaccard    -> SSE = 3660.3895, iters = 59

=== Mode: sse_increase ===
euclidean  -> SSE = 25414767689.9612, iters = 100
cosine     -> SSE = 686.2293, iters = 29
jaccard    -> SSE = 4239.9465, iters = 2

=== Mode: max_iter ===
euclidean  -> SSE = 25414767689.9612, iters = 100
cosine     -> SSE = 686.4356, iters = 100
jaccard    -> SSE = 3660.3895, iters = 100


### Q5

Q5: What are your summary observations or takeaways based on your algorithmic analysis?